# Operation Safe Passage: Agents

Agents are autonomous entities that interact with the OSP environment.  
Two types are supported:

- **UAV (Unmanned Aerial Vehicle)**: 
  - Moves across nodes.
  - Performs scans to estimate mine probability using configurable scanner models.
- **UGV (Unmanned Ground Vehicle)**:
  - Moves across nodes.
  - Detects and clears mines when encountered.

Agents are declared in `params.json` and are automatically created by the Mission Controller.


In [ ]:
import json
import pprint

from operation_safe_passage.controller.mission_controller import MissionController

## Minimal Configuration for Agents

To instantiate agents, `params.json` must define:

- **processing_params**
  - `UGV_traversal_time`: seconds per UGV move.
  - `UGV_clear_time`: seconds to clear a mine.
  - `UAV_traversal_time`: seconds per UAV move.

## Scanner Parameters

Each UAV scanner is defined in `scanner_params.scanners`.  
Parameters shape how accuracy and estimates are computed during scans:

- **visibility_scale (float)**: 
  Midpoint of the logistic function linking visibility percentage to accuracy.  
  Lower values make the scanner sensitive to small changes in visibility.

- **visibility_metric (int)**: 
  Steepness of the logistic curve. Higher values = sharper cutoff between poor/good visibility.

- **kappa (float)**: 
  Controls how strongly accuracy influences the mine probability estimate.  
  Larger values increase the separation between "mine" and "no mine."

- **noise_scale (float)**: 
  Governs variance of the probability estimate when accuracy is low.  
  Higher = noisier estimates.

- **noise_std (float)**: 
  Standard deviation of Gaussian noise added to accuracy.  
  Models random sensor error.

- **threshold (float)**: 
  Minimum accuracy required before the estimate becomes meaningful.  
  Below this threshold, scans output values closer to random.

- **time (int)**: 
  Seconds required to complete one scan.  
  Added to the mission clock each time a UAV scans.

- **terrain_coeff**
  - Mapping from terrain type to its effect on UAV scanner accuracy.


- **agents**
  - `uavs`: list of UAVs, each with a `name` and list of allowed scanners.
  - `num_ugvs`: number of UGVs to create.


In [ ]:
# Example configuration (presaved in config/params_agents.json) that defines:
# - Processing times for UAV and UGV actions
# - A single scanner ("default")
# - One UAV using the default scanner
# - One UGV
# - Basic terrain coefficient (Grassy)
params = {
    "processing_params": {
        "UGV_traversal_time": 20,   # seconds per move for UGV
        "UGV_clear_time": 60,       # seconds to clear a mine
        "UAV_traversal_time": 1     # seconds per move for UAV
    },
    "scanner_params": {
        "scanners": {
            "default": {
                "visibility_scale": 0.55,
                "visibility_metric": 6,
                "kappa": 3.0,
                "noise_scale": 3.0,
                "noise_std": 0.75,
                "threshold": 0.5,
                "time": 1             # seconds required for a scan
            }
        }
    },
    "agents": {
        "uavs": [
            {"name": "uav_alpha", "scanners": ["default"]}
        ],
        "num_ugvs": 1
    },
    "terrain_coeff": {"Grassy": 1.0}
}


## Initializing Agents

Agents are created when the Mission Controller is initialized.  
They are stored in `controller.agents`, in the order they appear in `params.json`.

Each agent has:
- `name`: string identifier (e.g., `"uav_alpha"`, `"UGV_0"`).
- `type`: `"UAV"` or `"UGV"`.
- `current_node`: the node where the agent is located.


In [ ]:
# Cell 7 - Initialize Controller (Code)
controller = MissionController(
    param_path="config/params_agents.json",
    network_path="config/premade_network.json",  # must exist from Map Creation step
    output_dir="output"
)

controller.reset()
[(agent.name, agent.type, agent.current_node) for agent in controller.agents]


## Controlling Agents

Each step requires a dictionary of actions:

- **Key**: agent object.  
- **Value**: dictionary of commands:
  - `move`: direction index (0–5) or string ("E", "NE", …).  
  - `scan`: scanner name or index (UAV only).  

Behavior:
- **UAVs**: perform movement and optional scans. Invalid moves are ignored.  
- **UGVs**: perform movement only. When entering a mined node, the mine is cleared, adding a time penalty.


In [ ]:
actions = {
    controller.agents[0]: {"move": "NE", "scan": "default"},  # UAV
    controller.agents[1]: {"move": "E"}                       # UGV
}

state = controller.step(actions)

print("Simulation time:", state["time"])
pprint.pprint(state["agents"])


## UAV Scanning Behavior

When a UAV scans a node:
- `metadata['uav_estimate']`: estimated mine probability [0,1].  
- `metadata['weight']`: updated to reflect scan result.  
- `metadata['scanned']`: set to `True`.  
- `inaccessible['uav_accuracy']`: hidden accuracy value (not visible to UAV).  
- Agent `scan_history`: updated with this estimate.  

These fields allow the controller and environment to incorporate UAV scan results into decision-making.

## UGV Mine-Clearing Behavior

When a UGV moves into a node:
- Marks `metadata['UGV_navigated'] = True`.  
- If a mine is present:
  - `metadata['mine'] = True`.  
  - Node is cleared.  
  - Mission time increases by `UGV_clear_time`.  
- `ugv_mine_detected`: stored on the node when a mine is encountered.

In [ ]:
state = controller.get_state()
print("Time:", state["time"])
pprint.pprint(state["agents"][0])  # details of UAV


## Summary

- **UAVs**
  - Move + scan using selected scanners.
  - Update node metadata with estimates.
  - Accuracy and estimates depend on environmental conditions and scanner parameters.

- **UGVs**
  - Move only.
  - Detect and clear mines with time penalty.

- **Mission State**
  - Provides simulation time, agent data, neighbors, and distances after each step.